# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [32]:
import numpy as np
import pandas as pd
import requests
import math
from scipy.stats import percentileofscore as score
import xlsxwriter
import warnings
warnings.filterwarnings('ignore')

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2482247876292,
 'week52high': 144.22,
 'week52low': 57.28,
 'week52change': 0.8406890134591988,
 'sharesOutstanding': 17370065119,
 'float': 0,
 'avg10Volume': 115164825,
 'avg30Volume': 113390658,
 'day200MovingAvg': 118.43,
 'day50MovingAvg': 134.17,
 'employees': 0,
 'ttmEPS': 3.36,
 'ttmDividendRate': 0.8064343492069717,
 'dividendYield': 0.005752809052807358,
 'nextDividendDate': '0',
 'exDividendDate': '2020-10-25',
 'nextEarningsDate': '2021-01-13',
 'peRatio': 43.43797801017498,
 'beta': 1.1587918344605614,
 'maxChangePercent': 54.08517205799035,
 'year5ChangePercent': 5.369876391980552,
 'year2ChangePercent': 2.8319378119234524,
 'year1ChangePercent': 0.8263001746140088,
 'ytdChangePercent': 0.07973283001951856,
 'month6ChangePercent': 0.5727339638220068,
 'month3ChangePercent': 0.255663268623341,
 'month1ChangePercent': 0.08726222253321742,
 'day30ChangePercent': 0.08517478550225453,
 'day5ChangePercent': 0.12191778157498609}

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
data['year1ChangePercent']

0.8263001746140088

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns=my_columns)
for symbol_string in symbol_strings:
    batch_api_call_url =f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe= final_dataframe.append(
        pd.Series(
        [
            symbol,
            data[symbol]['price'],
            data[symbol]['stats']['year1ChangePercent'],
            'N/A'
        ],
        index = my_columns),
        ignore_index = True
        )
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,122.910,0.427235,N/A
1,AAL,17.350,-0.454198,N/A
2,AAP,153.820,0.123712,N/A
3,AAPL,143.910,0.842996,N/A
4,ABBV,105.620,0.374842,N/A
...,...,...,...,...
500,YUM,103.140,0.0334799,N/A
501,ZBH,159.106,0.0796631,N/A
502,ZBRA,401.710,0.575294,N/A
503,ZION,45.690,0.025911,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [15]:
final_dataframe.sort_values('One-Year Price Return',ascending=False, inplace=True)
final_dataframe=final_dataframe[:50]
final_dataframe.reset_index(inplace= True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,78,CARR,40.10,2.3443,N/A
1,179,FCX,31.56,1.56545,N/A
2,275,LB,46.41,1.23415,N/A
3,23,ALB,178.78,1.19122,N/A
4,345,NVDA,555.60,1.18958,N/A
5,387,PYPL,254.00,1.13939,N/A
6,24,ALGN,559.97,0.979341,N/A
7,490,WST,303.55,0.930001,N/A
8,6,ABMD,356.80,0.928464,N/A
9,385,PWR,79.48,0.91141,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio:')
    
    try:
        float(portfolio_size)
    except ValueError:
        print('That is not a number! \nPlease try again:')
        portfolio_size = input('Enter the size of your portfolio:')
portfolio_input()
print(portfolio_size)

Enter the size of your portfolio:10000000
10000000


In [9]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i,'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i,'Price'])
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,122.910,0.427235,161
1,AAL,17.350,-0.454198,1141
2,AAP,153.820,0.123712,128
3,AAPL,143.910,0.842996,137
4,ABBV,105.620,0.374842,187
...,...,...,...,...
500,YUM,103.140,0.0334799,191
501,ZBH,159.106,0.0796631,124
502,ZBRA,401.710,0.575294,49
503,ZION,45.690,0.025911,433


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [80]:
hqm_columns =[
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

hqm_dataframe = pd.DataFrame(columns=hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url =f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
            pd.Series(
            [
                 symbol,
                 data[symbol]['price'],
                'NA',
                data[symbol]['stats']['year1ChangePercent'],
                'N/A',
                data[symbol]['stats']['month6ChangePercent'],
                'N/A',
                data[symbol]['stats']['month3ChangePercent'],
                'N/A',
                data[symbol]['stats']['month1ChangePercent'],
                'N/A',
                'N/A'
                
            ],
            index = hqm_columns),
            ignore_index = True
        )
hqm_dataframe    

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,119,NA,0.387978,N/A,0.232238,N/A,0.127246,N/A,0.0144451,N/A,N/A
1,AAL,17.12,NA,-0.372388,N/A,0.467228,N/A,0.475591,N/A,0.0587946,N/A,N/A
2,AAP,154.42,NA,0.074014,N/A,0.0109423,N/A,-0.0152204,N/A,-0.0793913,N/A,N/A
3,AAPL,146.54,NA,0.89327,N/A,0.526985,N/A,0.227762,N/A,0.079094,N/A,N/A
4,ABBV,107.28,NA,0.29569,N/A,0.087127,N/A,0.268765,N/A,0.00730533,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,104.89,NA,-0.00390638,N/A,0.104219,N/A,0.0474948,N/A,-0.0516445,N/A,N/A
501,ZBH,157.5,NA,0.0408564,N/A,0.147318,N/A,0.107641,N/A,0.0225838,N/A,N/A
502,ZBRA,396.69,NA,0.634718,N/A,0.436861,N/A,0.36315,N/A,0.0389488,N/A,N/A
503,ZION,44.73,NA,-0.022262,N/A,0.386574,N/A,0.426364,N/A,0.011087,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [81]:
time_periods =[
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
               ]
for row in hqm_dataframe.index:
    for time_period in time_periods:
        if hqm_dataframe.loc[row, f'{time_period} Price Return'] == None:
            hqm_dataframe.loc[row, f'{time_period} Price Return'] = 0

for row in hqm_dataframe.index:
    for time_period in time_periods:            
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[change_col],hqm_dataframe.loc[row,change_col])/100
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,119,NA,0.387978,0.851485,0.232238,0.615842,0.127246,0.50495,0.0144451,0.59802,N/A
1,AAL,17.12,NA,-0.372388,0.029703,0.467228,0.885149,0.475591,0.926733,0.0587946,0.792079,N/A
2,AAP,154.42,NA,0.074014,0.477228,0.0109423,0.170297,-0.0152204,0.144554,-0.0793913,0.0811881,N/A
3,AAPL,146.54,NA,0.89327,0.986139,0.526985,0.912871,0.227762,0.693069,0.079094,0.851485,N/A
4,ABBV,107.28,NA,0.29569,0.778218,0.087127,0.324752,0.268765,0.758416,0.00730533,0.556436,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,104.89,NA,-0.00390638,0.362376,0.104219,0.354455,0.0474948,0.29901,-0.0516445,0.192079,N/A
501,ZBH,157.5,NA,0.0408564,0.429703,0.147318,0.441584,0.107641,0.459406,0.0225838,0.631683,N/A
502,ZBRA,396.69,NA,0.634718,0.942574,0.436861,0.855446,0.36315,0.839604,0.0389488,0.712871,N/A
503,ZION,44.73,NA,-0.022262,0.332673,0.386574,0.825743,0.426364,0.891089,0.011087,0.574257,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [82]:
from statistics import mean 

for row in hqm_dataframe.index:
    momentum_percentiles =[]
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,119,NA,0.387978,0.851485,0.232238,0.615842,0.127246,0.50495,0.0144451,0.59802,0.642574
1,AAL,17.12,NA,-0.372388,0.029703,0.467228,0.885149,0.475591,0.926733,0.0587946,0.792079,0.658416
2,AAP,154.42,NA,0.074014,0.477228,0.0109423,0.170297,-0.0152204,0.144554,-0.0793913,0.0811881,0.218317
3,AAPL,146.54,NA,0.89327,0.986139,0.526985,0.912871,0.227762,0.693069,0.079094,0.851485,0.860891
4,ABBV,107.28,NA,0.29569,0.778218,0.087127,0.324752,0.268765,0.758416,0.00730533,0.556436,0.604455
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,104.89,NA,-0.00390638,0.362376,0.104219,0.354455,0.0474948,0.29901,-0.0516445,0.192079,0.30198
501,ZBH,157.5,NA,0.0408564,0.429703,0.147318,0.441584,0.107641,0.459406,0.0225838,0.631683,0.490594
502,ZBRA,396.69,NA,0.634718,0.942574,0.436861,0.855446,0.36315,0.839604,0.0389488,0.712871,0.837624
503,ZION,44.73,NA,-0.022262,0.332673,0.386574,0.825743,0.426364,0.891089,0.011087,0.574257,0.655941


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [83]:
hqm_dataframe.sort_values('HQM Score', ascending= False, inplace = True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(inplace= True, drop = True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,VIAC,58,NA,0.678596,0.950495,1.33927,0.99802,1.00872,0.988119,0.562335,1,0.984158
1,SIVB,462.5,NA,0.842784,0.980198,1.00594,0.992079,0.569697,0.948515,0.1957,0.962376,0.970792
2,ALB,175.42,NA,1.18945,0.994059,1.00139,0.990099,0.799715,0.980198,0.133077,0.916832,0.970297
3,DISCA,45.15,NA,0.458112,0.879208,1.14216,0.994059,1.23654,0.99604,0.554318,0.99802,0.966832
4,AMAT,101.91,NA,0.6947,0.954455,0.585032,0.934653,0.693858,0.974257,0.156669,0.940594,0.95099
5,MOS,27.62,NA,0.477109,0.89703,0.926472,0.982178,0.47398,0.924752,0.218196,0.970297,0.943564
6,GM,50.23,NA,0.505147,0.908911,0.929413,0.984158,0.421722,0.883168,0.187551,0.958416,0.933663
7,KEYS,158.28,NA,0.514655,0.914851,0.546337,0.918812,0.429726,0.89505,0.156717,0.942574,0.917822
8,LLY,216.15,NA,0.5419,0.932673,0.314042,0.746535,0.603842,0.960396,0.252798,0.988119,0.906931
9,FCX,27.07,NA,1.43201,0.99802,0.918748,0.980198,0.460902,0.918812,0.0417012,0.726733,0.905941


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [84]:
 portfolio_input()

Enter the size of your portfolio:1000000


In [85]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)
for i in hqm_dataframe.index:
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i, 'Price'])
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,VIAC,58,344,0.678596,0.950495,1.33927,0.99802,1.00872,0.988119,0.562335,1,0.984158
1,SIVB,462.5,43,0.842784,0.980198,1.00594,0.992079,0.569697,0.948515,0.1957,0.962376,0.970792
2,ALB,175.42,114,1.18945,0.994059,1.00139,0.990099,0.799715,0.980198,0.133077,0.916832,0.970297
3,DISCA,45.15,442,0.458112,0.879208,1.14216,0.994059,1.23654,0.99604,0.554318,0.99802,0.966832
4,AMAT,101.91,196,0.6947,0.954455,0.585032,0.934653,0.693858,0.974257,0.156669,0.940594,0.95099
5,MOS,27.62,724,0.477109,0.89703,0.926472,0.982178,0.47398,0.924752,0.218196,0.970297,0.943564
6,GM,50.23,398,0.505147,0.908911,0.929413,0.984158,0.421722,0.883168,0.187551,0.958416,0.933663
7,KEYS,158.28,126,0.514655,0.914851,0.546337,0.918812,0.429726,0.89505,0.156717,0.942574,0.917822
8,LLY,216.15,92,0.5419,0.932673,0.314042,0.746535,0.603842,0.960396,0.252798,0.988119,0.906931
9,FCX,27.07,738,1.43201,0.99802,0.918748,0.980198,0.460902,0.918812,0.0417012,0.726733,0.905941


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [86]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [87]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [88]:
column_formats = {
    'A':['Ticker', string_template],
    'B':['Price', dollar_template],
    'C':['Number of Shares to Buy', integer_template],
    'D':['One-Year Price Return', percent_template],
    'E':['One-Year Return Percentile', percent_template],
    'F':['Six-Month Price Return', percent_template],
    'G':['Six-Month Return Percentile', percent_template],
    'H':['Three-Month Price Return', percent_template],
    'I':['Three-Month Return Percentile', percent_template],
    'J':['One-Month Price Return', percent_template],
    'K':['One-Month Return Percentile', percent_template],
    'L':['HQM Score', percent_template]
}

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column( f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)


## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [89]:
writer.save()